In [22]:
# Imports
import yaml
from IPython.display import Markdown, display
import urllib
import json
import re

# Using the Bioportal API to retrieve the mappings of terms in the eNM ontology

# Motivation
The eNanoMapper imports terms from several ontologies. For example, CHEBI and NPO both contain the class `chemical substance`, but eNanoMapper only imports the one from CHEBI [CHEBI_59999](http://purl.obolibrary.org/obo/CHEBI_59999) and not [NPO_1973](http://purl.bioontology.org/ontology/npo#NPO_1973). How do these homonymous classes map to each other?

# Approach
Using the programmatic access to Bioportal, it is possible to retrieve the mapping data originating from using LOOM. This seems to be the only way to have access to this ontology mapping tool nowadays.
The mappings will be checked to relevant, upper(ish)-level classes, among all eNanoMapper ontology dependences.

In [86]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    dependencies = config["dependencies"]
    display(Markdown("**Dependencies** \n\n" + ", ".join([i for i in dependencies])))
    API_KEY = config["api_key"]
    classes = config["classes"]
print(classes)

**Dependencies** 

aopo, bao, bfo, bto, ccont, CHEBI, cheminf, chmo, cito, clo, efo, envo, fabio, go, iao, ncit, npo, oae, obcs, obi, pato, ro, sio, uberon, uo

{'chemical substance': 'http://purl.obolibrary.org/obo/CHEBI_59999', 'molecular entity': 'http://purl.obolibrary.org/obo/CHEBI_23367', 'material entity': 'http://purl.obolibrary.org/obo/BFO_0000040', 'nanomaterial': 'http://purl.bioontology.org/ontology/npo#NPO_199', 'nanoparticle': 'http://purl.bioontology.org/ontology/npo#NPO_707'}


Setting up a function to use the API ([documentation](https://data.bioontology.org/documentation#Mapping), [example response](https://data.bioontology.org/ontologies/CHEBI/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCHEBI_59999/mappings)):

In [ ]:
def mappings_lookup(dependencies, iri):
    f_iri = urllib.parse.quote(iri, safe='')
    for dependency in dependencies:
        rq = f"https://data.bioontology.org/ontologies/{dependency}/classes/{f_iri}/mappings"
        opener = urllib.request.build_opener()
        opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
        try:
            r = json.loads(opener.open(rq).read())
        except Exception as e:
            pass
        else:
            classes = [i["classes"] for i in r]
            matches = [classes[i] for i in range(len(classes))]
            matches = [matches[i][j]["@id"] for i in range(len(matches)) for j in range(0, len(matches[i]))]
            matches = [i for i in matches if i!=iri if any(re.match(dep, iri) for dep in dependencies)]
      
                


In [107]:
mappings = {term:mappings_lookup(dependencies, classes[term]) for term in classes}
with open("mappings.json", "w") as outfile:
    json.dump(mappings, outfile)


IRI: http://purl.obolibrary.org/obo/CHEBI_59999

- Dependency :CHEBI
- Matches from https://data.bioontology.org/ontologies/CHEBI/classes/http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FCHEBI_59999/mappings:
['http://vocabs.lter-europe.net/EnvThes/20799', 'http://sbmi.uth.tmc.edu/ontology/ochv#C0220806', 'http://purl.allotrope.org/ontologies/material#AFM_0001097', 'http://sweet.jpl.nasa.gov/2.3/matr.owl#ChemicalSubstance', 'http://purl.bioontology.org/ontology/npo#NPO_1973', 'http://www.w3.org/2001/sw/hcls/ns/transmed/TMO_0063', 'http://purl.jp/bio/4/id/200906093177020685', 'http://purl.bmicc.cn/ontology/ICD11CN/XM4FU3', 'http://www.co-ode.org/ontologies/galen#ChemicalSubstance', 'http://semanticscience.org/resource/SIO_011126', 'http://semanticscience.org/resource/SIO_011126', 'http://semanticscience.org/resource/SIO_011126', 'http://semanticscience.org/resource/SIO_011126', 'http://semanticscience.org/resource/SIO_011126', 'http://semanticscience.org/resource/SIO_011126', 'http://semanticsc